# 2D Monitoring Element Exploration

This notebooks provides some examples on how to access and visualize 2D monitoring elements using `cmsdials` and the visualization tools provided by `dqmexplore`.

## Imports

In [ ]:
# Setting up DIALS object and importing dqmexplore
import sys
sys.path.append("../src/")
import dqmexplore as dqme
dials = dqme.utils.setupdials.setup_dials_object_deviceauth()

In [ ]:
# Importing DIALS objects
# For more information on DIALS, please visit https://github.com/cms-DQM/dials-py
from cmsdials.filters import LumisectionHistogram1DFilters, LumisectionHistogram2DFilters

# Importing other useful packages
import numpy as np
import pandas as pd

In [ ]:
# The following function can be used to search for the exact name of the monitoring element you wish to analyze
dqme.utils.datautils.print_availMEs(
    dials, 
    contains="PixelPhase1/Tracks/PXBarrel/clusterposition_zphi_ontrack_PXLayer_", 
    dims=2
)

## 2D Monitoring Elements

In [ ]:
# Obtaininng ME data from DIALS
runnb = 380238
me__regex = "PixelPhase1/Tracks/PXBarrel/clusterposition_zphi_ontrack_PXLayer_."

data2D = dials.h2d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    max_pages=200
).to_pandas()

medata = dqme.medata.MEData(data2D)

print("Empty LSs for each ME:\n")
for me in medata.getMENames():
    print("{}:".format(me), medata.getEmptyLSs(me))

In [ ]:
# Getting trigger rate and plotting it
trig_rate = dqme.oms.get_rate(dials, runnb)
trig_rate_plot = dqme.oms.plot_rate(
    trig_rate, 
    fig_title="Trigger Rate ({})".format(runnb), 
    norm=False, 
    show=True
)

In [ ]:
# Defining plot features and plotting
ax_labels = [
    dict(x="SignedModuleCoord", y="SignedLadderCoord")
] * 4

fig_title = f"Pixel Barrel Digi Occupancy Normalized by Trigger Rate (Run {runnb})"

fig2D = dqme.interplt.plot2DMEs(
    medata,
    fig_title=fig_title,
    ax_labels=ax_labels,
    # trigger_rates=trig_rate,
    height=800,
    width=1250,
    hspace=0.2,
    show=True
)

The figure constructed above can be further modified using `plotly`.

In [ ]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add modifications to fig2D here.


And you can also save the figure by running the folling cell.

In [ ]:
# You can set show=False and export the figure to an HTML file by also running the code below
from plotly.offline import plot
plot(fig2D, filename=f"./fig2d.html")